In [6]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, Button, Dropdown, HBox, Label, Output, VBox
from ipycanvas import Canvas
from IPython.display import display, clear_output

rng = np.random.default_rng()
power_limit = np.arange(0, 100, 1) / 100
min_in_day = 24*60
minute_power = None
custom = False

# Canvas setup
canvas_width, canvas_height = 400, 200
canvas = Canvas(
    width=canvas_width,
    height=canvas_height,
    sync_image_data=True,
    layout={'border': '1px solid black', 'width': f'{canvas_width}px'}
)
is_drawing, last_x, last_y = False, 0, 0

@canvas.on_mouse_down
def mouse_down(x, y):
    global is_drawing, last_x, last_y
    is_drawing, last_x, last_y = True, x, y
@canvas.on_mouse_move
def mouse_move(x, y):
    global is_drawing, last_x, last_y
    if is_drawing:
        canvas.stroke_line(last_x, last_y, x, y)
        last_x, last_y = x, y
@canvas.on_mouse_up
def mouse_up(x, y):
    global is_drawing
    is_drawing = False
@canvas.on_mouse_out
def mouse_out(x, y):
    global is_drawing
    is_drawing = False

def setup_canvas():
    canvas.clear()
    canvas.fill_style = "rgba(0, 0, 0, 0)"  # Fully transparent background
    canvas.fill_rect(0, 0, canvas_width, canvas_height)
    # Draw grid and axes
    canvas.line_width = 1
    canvas.stroke_style = "rgba(1, 1, 1, 0.5)"
    canvas.stroke_line(0, canvas_height, canvas_width, canvas_height)
    canvas.stroke_line(0, canvas_height, 0, 0)
    canvas.stroke_style = "rgba(15, 15, 15, 0.5)"
    for i in range(1, 10):
        y = canvas_height / 10 * i
        canvas.stroke_line(0, y, canvas_width, y)
    for i in range(1, 10):
        x = canvas_width / 10 * i
        canvas.stroke_line(x, 0, x, canvas_height)
    canvas.stroke_style = "rgba(0, 0, 0, 1)"
    canvas.line_width = 3

def on_capture_button_click(b):
    global minute_power, custom
    img_data = canvas.get_image_data()
    minute_power = np.zeros(min_in_day)
    for i in range(min_in_day):
        cx = int(i / min_in_day * canvas_width)
        for cy in range(canvas_height):
            if img_data[cy, cx, 3] > 200:
                minute_power[i] = (canvas_height - cy) / canvas_height
                break
    minute_power = np.tile(minute_power, 365)
    custom = True
    update_distribution('Custom')

def on_clear_button_click(b):
    setup_canvas()

btn_capture = Button(description="Use these values")
btn_clear = Button(description="Clear")
btn_capture.on_click(on_capture_button_click)
btn_clear.on_click(lambda b: setup_canvas())

def update_distribution(distribution):
    global minute_power, custom, graphs
    if distribution == 'Custom' and not custom:
        canvasbox.layout.display = 'block'
        setup_canvas()
        return
    elif distribution != 'Custom':
        if custom:
            canvasbox.layout.display = 'none'
            custom = False
        match distribution:
            case 'Uniform':
                minute_power = rng.uniform(0, 1, 365*24*60)
            case 'Gaussian':
                minute_power = rng.normal(0.5, 0.1, 365*24*60)
                minute_power = np.clip(minute_power, 0, 1)

    tot = max(np.sum(minute_power), 1)
    hist, bin_edges = np.histogram(minute_power, bins=100, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    hist = hist / np.sum(hist)
    #cdf = np.cumsum(hist)
    curtailed = [np.sum(np.maximum(0, minute_power - t)) / tot for t in power_limit]
    graphs.clear_output()
    with graphs:
        plt.figure(figsize=(15, 5))
        plt.subplot(131)
        plt.plot(bin_centers, hist)
        #plt.plot(bin_centers, cdf)
        plt.xlabel('Power level')
        plt.ylabel('Relative frequency (PDF)')
        plt.ylim(0)
        plt.xlim(0)
        plt.title('Distribution')
        
        plt.subplot(132)
        plt.plot(minute_power[:1440])
        plt.xlabel('Minutes')
        plt.ylabel('Power')
        plt.ylim(0, 1)
        plt.title('24 Hours of Power Data')
        
        plt.subplot(133)
        plt.plot(power_limit, curtailed)
        plt.xlabel('Power Limit')
        plt.ylabel('Curtailed Energy Fraction')
        plt.ylim(0, 1)
        plt.title('Curtailment')
        
        plt.tight_layout()
        plt.show()

def distribution_onchange(change):
    update_distribution(change.new)
dropdown_default = 'Uniform'
sel_distribution = Dropdown(
    options=['Uniform', 'Gaussian', 'Custom'],
    value=dropdown_default,
    description='Distribution:',
)
graphs = Output()
canvasbox = VBox([ Label(value="Draw power generation over 24h"),
                   canvas,
                   HBox([btn_capture, btn_clear])
                 ])
canvasbox.layout.display = 'none'
sel_distribution.observe(distribution_onchange, names='value')
update_distribution(dropdown_default)
display(VBox([sel_distribution, graphs, canvasbox]))
